<a href="https://colab.research.google.com/github/johhan27/DS-Unit-2-Linear-Models/blob/master/LS_DS_212_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Module Project: Regression II

In this project, you'll continue working with the New York City rent dataset you used in the last module project.

## Directions

The tasks for this project are as follows:

- **Task 1:** Import `csv` file using `wrangle` function.
- **Task 2:** Conduct exploratory data analysis (EDA), and modify `wrangle` function to engineer two new features.
- **Task 3:** Split data into feature matrix `X` and target vector `y`.
- **Task 4:** Split feature matrix `X` and target vector `y` into training and test sets.
- **Task 5:** Establish the baseline mean absolute error for your dataset.
- **Task 6:** Build and train a `Linearregression` model.
- **Task 7:** Calculate the training and test mean absolute error for your model.
- **Task 8:** Calculate the training and test $R^2$ score for your model.
- **Stretch Goal:** Determine the three most important features for your linear regression model.

**Note**

You should limit yourself to the following libraries for this project:

- `matplotlib`
- `numpy`
- `pandas`
- `sklearn`

# I. Wrangle Data

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [16]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    
    # Remove the most extreme 1% prices,
    # the most extreme .1% latitudes, &
    # the most extreme .1% longitudes
    df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
            (df['price'] <= np.percentile(df['price'], 99.5)) & 
            (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
            (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
            (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
            (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
    df.index = pd.to_datetime(df['created'])
    df = df.dropna()
    return df

filepath = DATA_PATH + 'apartments/renthop-nyc.csv'

**Task 1:** Add the following functionality to the above `wrangle` function.

- The `'created'` column will parsed as a `DateTime` object and set as the `index` of the DataFrame. 
- Rows with `NaN` values will be dropped.

Then use your modified function to import the `renthop-nyc.csv` file into a DataFrame named `df`.

In [39]:
df = wrangle(filepath)
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Task 2:** Using your `pandas` and dataviz skills decide on two features that you want to engineer for your dataset. Next, modify your `wrangle` function to add those features. 

**Note:** You can learn more about feature engineering [here](https://en.wikipedia.org/wiki/Feature_engineering). Here are some ideas for new features:

- Does the apartment have a description?
- Length of description.
- Total number of perks that apartment has.
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths).

In [40]:
df['vip_lat'] = [1 if (i > 40.7) and (i < 40.8) else 0 for i in df['latitude']]
df['vip_lon'] = [1 if (i > -74.02) and (i < -73.95) else 0 for i in df['longitude']] 

In [41]:
#df[df['price']>8000].sort_values(by='longitude').tail()
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,vip_lat,vip_lon
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2016-06-12 12:19:27,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2016-04-17 03:26:41,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2016-04-18 02:22:02,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2016-04-28 01:32:41,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
x_cols = [i for i in df.describe().columns]
x_cols.remove('price')
print (x_cols)

['bathrooms', 'bedrooms', 'latitude', 'longitude', 'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed', 'doorman', 'dishwasher', 'no_fee', 'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space', 'vip_lat', 'vip_lon']


In [89]:
# latitude and longitude are not helpful for the sum we plan to do
x_cols.remove('latitude')
x_cols.remove('longitude')
print(x_cols)

['bathrooms', 'bedrooms', 'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed', 'doorman', 'dishwasher', 'no_fee', 'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space', 'vip_lat', 'vip_lon']


In [90]:
over_mean_minus_below_mean = df.loc[df['price'] > df['price'].mean()][x_cols].mean() - df.loc[df['price'] < df['price'].mean()][x_cols].mean()
print(over_mean_minus_below_mean)

bathrooms               0.496449
bedrooms                1.162738
elevator                0.182258
cats_allowed            0.062896
hardwood_floors         0.071248
dogs_allowed            0.070262
doorman                 0.231061
dishwasher              0.196722
no_fee                  0.110532
laundry_in_building    -0.008151
fitness_center          0.180422
pre-war                -0.022497
laundry_in_unit         0.175257
roof_deck               0.072592
outdoor_space           0.087799
dining_room             0.111545
high_speed_internet     0.047965
balcony                 0.057423
swimming_pool           0.046876
new_construction        0.020337
terrace                 0.046888
exclusive              -0.001772
loft                   -0.011515
garden_patio            0.033202
wheelchair_access       0.017852
common_outdoor_space    0.005210
vip_lat                 0.146774
vip_lon                 0.189058
dtype: float64


In [91]:
df_over_mean_minus_below_mean = pd.DataFrame(over_mean_minus_below_mean) #made a df to manipulate it b

In [100]:
#we take the positive significant values as top columns and less relevant as lower columns (no negative values)
index_top_cols = list (df_over_mean_minus_below_mean[df_over_mean_minus_below_mean[0] > 0.1].index)
print ("top", index_top_cols)
index_lower_cols = list (df_over_mean_minus_below_mean[(df_over_mean_minus_below_mean[0] < 0.1) & (df_over_mean_minus_below_mean[0] > 0)].index)
print ("lower", index_lower_cols)
#creating our index column here with the top columns
df['top_index'] = df[index_top_cols].sum(axis=1)
df['lower_index'] = df[index_lower_cols].sum(axis=1)  

top ['bathrooms', 'bedrooms', 'elevator', 'doorman', 'dishwasher', 'no_fee', 'fitness_center', 'laundry_in_unit', 'dining_room', 'vip_lat', 'vip_lon']
lower ['cats_allowed', 'hardwood_floors', 'dogs_allowed', 'roof_deck', 'outdoor_space', 'high_speed_internet', 'balcony', 'swimming_pool', 'new_construction', 'terrace', 'garden_patio', 'wheelchair_access', 'common_outdoor_space']


# II. Split Data

**Task 3:** Split your DataFrame `df` into a feature matrix `X` and the target vector `y`. You want to predict `'price'`.

**Note:** In contrast to the last module project, this time you should include _all_ the numerical features in your dataset.

In [93]:
print (x_cols)

['bathrooms', 'bedrooms', 'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed', 'doorman', 'dishwasher', 'no_fee', 'laundry_in_building', 'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck', 'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony', 'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft', 'garden_patio', 'wheelchair_access', 'common_outdoor_space', 'vip_lat', 'vip_lon']


In [105]:
features= ['top_index', 'bathrooms']
target = ['price']

X = df[features]
y = df[target]


**Task 4:** Split `X` and `y` into a training set (`X_train`, `y_train`) and a test set (`X_test`, `y_test`).

- Your training set should include data from April and May 2016. 
- Your test set should include data from June 2016.

In [106]:
#df.loc['2014-01-01':'2014-02-01']
X_train, y_train = X.loc['2016-04-01':'2016-06-01'], y.loc['2016-04-01':'2016-06-01']
X_test, y_test = X.loc['2016-06-01':], y.loc['2016-06-01':]

# III. Establish Baseline

**Task 5:** Since this is a **regression** problem, you need to calculate the baseline mean absolute error for your model. First, calculate the mean of `y_train`. Next, create a list `y_pred` that has the same length as `y_train` and where every item in the list is the mean. Finally, use `mean_absolute_error` to calculate your baseline.

In [65]:
from sklearn.metrics import mean_absolute_error

baseline_mae = mean_absolute_error(y['price'], [y['price'].mean() for i in range(len(y))]) 
print('Baseline MAE:', baseline_mae)

Baseline MAE: 1203.1899321076858


# IV. Build Model

**Task 6:** Build and train a `LinearRegression` model named `model` using your feature matrix `X_train` and your target vector `y_train`.

In [107]:
# Step 1: Import predictor class
from sklearn.linear_model import LinearRegression

# Step 2: Instantiate predictor
model = LinearRegression()

# Step 3: Fit predictor on the (training) data
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# V. Check Metrics

**Task 7:** Calculate the training and test mean absolute error for your model.

In [108]:
training_mae = mean_absolute_error( y_train, model.predict(X_train))
test_mae = mean_absolute_error( y_test, model.predict(X_test))

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

Training MAE: 767.2839267084853
Test MAE: 771.382203939385


**Task 8:** Calculate the training and test $R^2$ score for your model.

In [118]:
from sklearn.metrics import r2_score
training_r2 = r2_score(y_train, model.predict(X_train))
test_r2 = r2_score(y_test, model.predict(X_test))

print('Training R2:', training_r2)
print('Test R2:', test_r2)

Training R2: 0.5612175191297611
Test R2: 0.5685939566761196


# VI. Communicate Results

**Stretch Goal:** What are the three most influential coefficients in your linear model? You should consider the _absolute value_ of each coefficient, so that it doesn't matter if it's positive or negative.

In [115]:
model.intercept_

array([-112.56772459])

In [116]:
model.coef_

array([[ 229.3990919 , 1770.60921369]])

In [117]:
beta0 = model.intercept_
beta1, beta2 = model.coef_[0]
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

y = [-112.56772459] + 229.39909189794557x1 + 1770.6092136888008x2


In [ ]:
#every unit of "top_index" equals to $229.4 and bathroom to $1770
#Also we could see that over 56% of the variance of "price" is explained by our features for both training and test data